In [14]:
import os
import random
import pandas as pd
from tqdm import tqdm
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from transformers import BlipProcessor, BlipForConditionalGeneration

In [11]:
test_csv = 'test.csv'
test_images = 'test/test'
processor_path = "kaggle/working/blip-finetuned"
model_path = "kaggle/working/blip-finetuned/checkpoint-8013"

In [12]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Kullanılan cihaz:", device)

Kullanılan cihaz: cuda


In [16]:
# ------------------------ Caption Üretimi ------------------------ #
test_df = pd.read_csv(test_csv)
test_df["image_id"] = test_df["image_id"].astype(str)

caption_processor = BlipProcessor.from_pretrained(processor_path, local_files_only=True)
caption_model = BlipForConditionalGeneration.from_pretrained(model_path, local_files_only=True).to(device)
caption_model.eval()

captions = []

print("Caption üretimi başlıyor...")
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    image_path = os.path.join(test_images, str(row["image_id"]) + ".jpg")
    image = Image.open(image_path).convert("RGB")
    
    inputs = caption_processor(image, return_tensors="pt").to(device)

    with torch.no_grad():
        output = caption_model.generate(
            **inputs,
            max_length=128,
            num_beams=5,           # Beam search ile daha kaliteli captionlar
            no_repeat_ngram_size=2,
            early_stopping=True
        )

    caption = caption_processor.decode(output[0], skip_special_tokens=True)
    captions.append(caption)

submission_df = pd.DataFrame({
    "image_id": test_df["image_id"],
    "caption": captions
})
submission_df.to_csv("/kaggle/working/submission.csv", index=False)
print("Captionlar başarıyla kaydedildi!")

Caption üretimi başlıyor...


100%|██████████| 3771/3771 [11:44:07<00:00, 11.20s/it]      


OSError: Cannot save file into a non-existent directory: '\kaggle\working'

In [17]:
submission_df.to_csv("kaggle/working/submission.csv", index=False)
print("Captionlar başarıyla kaydedildi!")

Captionlar başarıyla kaydedildi!
